## Libraries

In [7]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input,Output, automl, command
from azure.ai.ml.constants import AssetTypes

## Login Azure

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## Conection instance

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: .\config.json


## Automate machine learning pipeline

- Tested Models:
    - Logistic regression
    - Light gbm
    - Random forest

In [19]:
# Define pipeline
@pipeline(
    description="AutoML Classification Pipeline",
    )
def automl_classification(
    classification_train_data,
    classification_validation_data
):
    # define the automl classification task with automl function
    classification_node = automl.classification(
        training_data=classification_train_data,
        validation_data=classification_validation_data,
        target_column_name="default",
        primary_metric="accuracy",
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    # set limits and training
    classification_node.set_limits(max_trials=1)
    classification_node.set_training(
        enable_stack_ensemble=False,
        enable_vote_ensemble=False,
        allowed_training_algorithms=['logistic_regression','light_gbm','random_forest']
    )

    command_func = command(
        inputs=dict(
            automl_output=Input(type="mlflow_model")
        ),
        command="ls ${{inputs.automl_output}}",
        environment="azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest"
    )
    show_output = command_func(automl_output=classification_node.outputs.best_model)

## Train & Test location

In [21]:
URI_TRAIN_PATH = "YOUR_URI_PATH_TO_MLTABLE_IN_AZURE"
URI_TEST_PATH = "YOUR_URI_PATH_TO_MLTABLE_IN_AZURE"

## Submit & Execute Pipeline

In [22]:
import logging
try:
    pipeline_job = automl_classification(
        classification_train_data=Input(path=URI_TRAIN_PATH, type=AssetTypes.MLTABLE),
        classification_validation_data=Input(path=URI_TEST_PATH, type=AssetTypes.MLTABLE),
    )
    
    # Set pipeline level compute
    pipeline_job.settings.default_compute = "juan-risk"

    # Submit the pipeline job
    returned_pipeline_job = ml_client.jobs.create_or_update(
        pipeline_job,
        experiment_name="project-codigo-facilito"
    )

    logging.info("Pipeline job submitted successfully: %s", returned_pipeline_job)
except Exception as e:
    logging.error("An error occurred while creating or updating the pipeline job: %s", e)
    raise